如果按照顺序看完前面的章节, 现在已经清楚什么是`package.json` `package-lock.json` `node_modules` `app.js`这几个文件, 还清楚了`express`只是个module, 如何使用`npm`安装modules, 并且理解了`nodejs`只是javascript的一个runtime environment, modules在nodejs的重要意义. 

下面我们就开始学习nodejs里面的express module, 在IDEA创建了express项目后, 出现了一大堆文件夹, 所以第一步要弄明白这些文件和文件夹都是用来放什么代码的, 其中根目录除了上面提到的四个文件之外, 还有几个重要的文件夹, 分别是 `views`, `controllers`, `models`. `views`里面存的应该是html网页之类的东西, 而真正的逻辑, 如何把数据存到数据库, 如何返回数据, 都存在了`controllers`下面, . 而`models`就是放每个数据库实体的代码, 就是orm, 比如一个猫, 有年龄, 名字吗体重, 就像是个类, 这也就是所谓的建立模型. 

结合实际项目去看结构, 体会最深刻, 可以在github随便找个exress的项目看看,最好是有rest api的.

# Create An Express App

---

文章名虽然是Create An Express App, 但要清楚express只是一个module. 

[Building a REST API with Node and Express](https://stackabuse.com/building-a-rest-api-with-node-and-express/)


## 1. 安装MongoDB

关于如何安装使用MongoDB, 在`MyNotes/database/`文件夹中有记录.


## 2. Modules Need to be Installed

```shell
npm install express

npm instal mongoose

# body-parser和cors都是middleware, 可以去middleware那一节文章查看定义
# 这里注意一下, body-parser 不用装了, 因为4.16.0版本的express又把它加进去了
# 用的时候app.use(express.json()) 即可, 具体看下文代码
# npm install body-parser

npm install cors
```

## 3. Routing

具体endpoint设计参照endpoint design那一节的内容,应该就是上一节.

```js
const app = require('express')();

app.get('/temperature', (req, res) => {
    res.send('data from temperature');
    console.log(Object.keys(req.query).length);
    console.log(req.query.date);
    console.log(req.query.num);
});

app.post('/temperature', (req, res)=>{});

app.listen(port, () => console.log('start listening...'));
```

## 3.  连接MongoDB 并运行

查看下默认的port没有记录, 谷歌了一下是`27017`

```shell
cat /opt/homebrew/etc/mongod.conf 

systemLog:
  destination: file
  path: /opt/homebrew/var/log/mongodb/mongo.log
  logAppend: true
storage:
  dbPath: /opt/homebrew/var/mongodb
net:
  bindIp: 127.0.0.1, ::1
  ipv6: true
```

**app.js**:
```js
/** Import modules **/
const express = require('express');
// If only use: const mongoose = require('mongoose');
// There will be an error: unresolved function or method connect() mongoose
// on jetbrains IDEA
const MODEL_PATH = './node_modules/';
const mongoose = require(MODEL_PATH + 'mongoose');
const cors = require('cors');
const {getTemperature, createTemperature} = require('./controllers/temperature');

/** Add middleware functions **/
const app = express();
app.use(express.json());
app.use(cors());

/** Routing, design endpoint **/
app.get('/temperature', getTemperature);
app.post('/temperature', createTemperature);

app.get('/humidity', (req, res) => {
    res.send('data from temperature');
    console.log(Object.keys(req.query).length);
    console.log(req.query.date);
    console.log(req.query.num);
});

/** Connect database and run server**/
main().catch(err => console.log(err));

async function main() {
    await mongoose.connect('mongodb://localhost:27017/greenhouse');
    app.listen(3000, () => console.log('start listening...'));
}
```


**model/temperature.js**:
```js
const mongoose = require('mongoose');

const temperatureSchema = new mongoose.Schema({
    value: Number,
    date: Date
});

// 非常重要, 必须是三个参数, 我一直看官网说的俩参数, 结果一直无法获得数据
// 三个参数分别是: model name, schema, collection name
const Temperature = mongoose.model('temperatureModel', temperatureSchema,'temperature');

module.exports = Temperature;
```

**controller/temperature.js**:
```js
const Temperature = require('../models/temperature');

const getTemperature = async (req, res) => {
    try {
        const temperature = await Temperature.find();

        res.status(200).json(temperature);
    } catch (error) {
        res.status(404).json({ message: error.message });
    }
}

const createTemperature = async (req, res) => {
    const { value, date } = req.body;
    const newTemperature = new Temperature({ value, date });
    try {
        await newTemperature.save();

        res.status(201).json(newTemperature);
    } catch (error) {
        res.status(409).json({ message: error.message });
    }
}

module.exports = {
    getTemperature,
    createTemperature
}
```

# Express App Project Structure
---

